In [13]:
import matplotlib.pyplot as plt
import numpy as np
import torch as th

from cnn import CNN, createLossAndOptimizer
from torch.utils.data import DataLoader
from tqdm import tqdm

from imgdata import DefaultTrainSet

In [14]:
%matplotlib inline

if th.cuda.is_available():
  # Make CuDNN Determinist
  th.backends.cudnn.deterministic = True

# Define default device, we should use the GPU (cuda) if available
device = th.device("cuda" if th.cuda.is_available() else "cpu")
device

device(type='cpu')

### Set Constants
define learning rate as a list of values, and the number of epochs and batch size

In [15]:
learning_rates = [1e-2, 1e-3, 1e-4, 1e-5]
batch_size = 16
num_epochs = 20

### Load data
using the DefaultTrainSet class from imgdata.py

In [16]:
train_set = DefaultTrainSet()

train_dataloader = th.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)

### Define the training function

In [17]:
def train(net, batch, n_epochs, lr, train_loader):
    """
    Train a neural network and print statistics of the training

    :param train_loader: loader for the training data
    :param net: (PyTorch Neural Network)
    :param batch: (int)
    :param n_epochs: (int)  Number of iterations on the training set
    :param lr: (float) learning rate used by the optimizer
    """
    print("---------- HYPERPARAMETERS ----------")
    print("batch_size=", batch)
    print("n_epochs=", n_epochs)
    print("learning_rate=", lr)
    print("-" * 40)
    accuracies = []
    losses = []
    criterion, optimizer = createLossAndOptimizer(net, lr)
    # Move model to gpu if possible
    net = net.to(device)
    for epoch_count, epoch in enumerate(range(n_epochs)):
        # Set the model to training mode
        net.train()
        # Initialize the variables to compute the average loss
        total_loss = 0.00
        avg_loss = 0.00
        total_correct = 0
        pbar = tqdm(train_loader, leave=False)
        # Iterate over the training set
        for i, data in enumerate(pbar):

            # Get the inputs
            inputs, labels = data['imNorm'], data['label']
            # Move them to the correct device
            inputs = inputs.to(device)
            labels = labels.to(device)
            net = net.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = net(inputs)
             # Calculate Loss: softmax --> cross entropy loss
            loss = criterion(outputs, labels)
            
            _, preds = th.max(outputs, 1)

            # Getting gradients w.r.t. parameters
            loss.backward()

            # Updating parameters
            optimizer.step()

            # Update the running variables
            total_loss += loss.item()
            avg_loss = total_loss / (i + 1)
            total_correct += th.sum(preds == labels.data)
            pbar.set_description("Epoch: {}/{}".format(epoch_count + 1, n_epochs))
            pbar.set_postfix(avergae_loss=avg_loss, running_accuracy=th.sum(preds == labels).item() / float(labels.size(0)))
            # save the model
            model_name = f"Outputs/models/model_{num_epochs}_epochs_learning_rate_{lr}.pth"
            th.save(net.state_dict(), model_name)

        # Calculate the average loss and accuracy of the epoch
        losses.append(total_loss/len(train_loader))
        accuracies.append((total_correct/train_loader.sampler.num_samples).item())
        # print losses and accuracies in the last epoch
        if epoch_count == n_epochs - 1:
            print("---------- FINAL RESULTS ----------")
            print("Losses fot every epoch:", losses)
            print("Average accuracy for each epoch: {}".format(accuracies))
            print("-" * 40)
    return losses, accuracies

In [19]:
losses_list = []
accuracies_list = []
for learning_rate in learning_rates:
    cnn = CNN()
    print("Training with learning rate {}\n".format(learning_rate))
    loss, accu = train(cnn, batch_size, n_epochs=num_epochs, lr=learning_rate, train_loader=train_dataloader)
    losses_list.append(loss)
    accuracies_list.append(accu)
    print("-"*40)

Training with learning rate 0.01

---------- HYPERPARAMETERS ----------
batch_size= 16
n_epochs= 20
learning_rate= 0.01
----------------------------------------


---------- FINAL RESULTS ----------
Losses fot every epoch: [105.2439342953942, 23.786831205541436, 10.40375535867431, 7.107119262218475, 8.709446325898107, 2.0403839888093773, 2.0951686815363826, 0.36044958136675764, 0.47943789798604347, 0.29616873867317656, 0.1300860537049295, 1.066671657675201, 1.8639609571694224, 0.8999131580064634, 0.6043057237497929, 0.17278097692625038, 0.1085460292505663, 0.4514570550560883, 0.44791949637371986, 0.7925233316454975]
Average accuracy for each epoch: [0.41999998688697815, 0.691428542137146, 0.7200000286102295, 0.7942857146263123, 0.7657142877578735, 0.9085714221000671, 0.9171428680419922, 0.9657142758369446, 0.9571428298950195, 0.9571428298950195, 0.9857142567634583, 0.9371428489685059, 0.9114285707473755, 0.9399999976158142, 0.954285740852356, 0.9885714054107666, 0.9885714054107666, 0.9742857217788696, 0.9685714244842529, 0.9428571462631226]
----------------------------------------
----------------------------------------
Training with learning r

---------- FINAL RESULTS ----------
Losses fot every epoch: [13.611489805308254, 2.802003250880675, 1.6827044080604205, 1.1672562899237329, 0.5217582395868952, 0.36627430507197994, 0.49466115269344546, 0.6013061869509163, 0.6550813368735362, 0.3423406942807865, 0.1652563704895494, 0.660713486645231, 0.32787752254630415, 0.04587854106305285, 0.22528496175611015, 0.11573040578351236, 0.036868374941601084, 0.05645058883005214, 0.09483473685211292, 0.05800028894595691]
Average accuracy for each epoch: [0.4514285624027252, 0.7200000286102295, 0.7742857336997986, 0.8257142901420593, 0.9142857193946838, 0.9457142949104309, 0.9228571653366089, 0.9171428680419922, 0.9085714221000671, 0.9371428489685059, 0.9685714244842529, 0.9142857193946838, 0.9485714435577393, 0.9857142567634583, 0.9742857217788696, 0.9800000190734863, 0.991428554058075, 0.9885714054107666, 0.9828571677207947, 0.9828571677207947]
----------------------------------------
----------------------------------------
Training with l

---------- FINAL RESULTS ----------
Losses fot every epoch: [1.2380337796427987, 0.4295810457657684, 0.1643308876928958, 0.08902897973629562, 0.05043892545456236, 0.032433065712790594, 0.024996084880761126, 0.01932518222284588, 0.018553614785725422, 0.014533033403991298, 0.011886080938645384, 0.00932889631737701, 0.008116199444471435, 0.006587355302392759, 0.006144530266862024, 0.005237370792945678, 0.005168165556493808, 0.005417329153384675, 0.004282652495682917, 0.004219262522052635]
Average accuracy for each epoch: [0.5714285969734192, 0.8342857360839844, 0.9628571271896362, 0.9885714054107666, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
----------------------------------------
----------------------------------------
Training with learning rate 1e-05

---------- HYPERPARAMETERS ----------
batch_size= 16
n_epochs= 20
learning_rate= 1e-05
----------------------------------------


---------- FINAL RESULTS ----------
Losses fot every epoch: [1.409951090812683, 0.9138424938375299, 0.6942490515383807, 0.575256044214422, 0.4797831340269609, 0.4205977666107091, 0.36472240361300384, 0.3150969642129811, 0.27685957469723443, 0.26010610637339676, 0.2218981280245564, 0.19456915455785664, 0.17399421605196866, 0.16082928905432875, 0.14961659468033098, 0.13195123388008637, 0.11791059950535948, 0.11301942677660422, 0.10479728030887517, 0.09931005639108745]
Average accuracy for each epoch: [0.3942857086658478, 0.7342857122421265, 0.7942857146263123, 0.8428571224212646, 0.8742856979370117, 0.8942857384681702, 0.9228571653366089, 0.9399999976158142, 0.9571428298950195, 0.9485714435577393, 0.9714285731315613, 0.9828571677207947, 0.991428554058075, 0.991428554058075, 0.9857142567634583, 0.9942857027053833, 1.0, 0.9942857027053833, 0.991428554058075, 0.9942857027053833]
----------------------------------------
----------------------------------------


In [ ]:
# plot the losses of training against the number of epochs
def plot_loss(losses, lr):
    plt.figure(figsize=(10,5))
    for _i, _loss in enumerate(losses):
        plt.plot(_loss, label="learning rate: {}".format(lr[_i]))
    plt.title("Loss vs. Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.grid(True)
    plt.legend()
    plt.show()

# plot the accuracy of training against the number of epochs
def plot_accuracy(accuracies, lr):
    plt.figure(figsize=(10,5))
    for _i, _accu in enumerate(accuracies):
        plt.plot(_accu, label="learning rate: {}".format(lr[_i]))
    plt.title("Accuracy vs. Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.grid(True)
    plt.legend()
    plt.show()

In [ ]:
# save the losses and accuracies in a numpy file
train_history = {'loss': np.array(losses_list), 'accuracy': np.array(accuracies_list)}
np.save('Outputs/models/train_history.csv', train_history)

In [ ]:
train_history = np.load('Outputs/models/train_history.csv', allow_pickle=True).item()
losses = train_history['loss']
accuracies = train_history['accuracy']

#plot the losses and accuracies
plot_loss(losses, learning_rates)
plot_accuracy(accuracies, learning_rates)